In [29]:
from pathlib import Path

import pandas as pd

pd.options.display.max_columns = None

# The importance of sandboxes

I've worked with SQL regularly over the past decade, but over the first half of that decade (and despite extensive formal knowledge of normalization, data modeling, and other database-related topics) I essentially only used SQL to extract data and use Python or R to do analysis. I only had access to production databases and I didn't know enough about how database management system work to risk experiments, and as a result, I learned at a glacial pace. But in 2018, I stumbled into a project interesting enough to motivate me to install PostGIS on a personal machine and freed from the fear of accidentally taking down a production system (and the power of a superuser), I was able to lift up the hood and see how the machine worked, and this enabled me to learn and build with Postgres + PostGIS (a geospatial Postgres extension) far faster.

In this post, I show how to use Docker to set up a PostGIS database and experiment with it.

## What is Docker

Docker and Docker Compose[1^] enable you to import and run complicated applications in just a few lines of code.

The internals of docker are really interesting, but for this post, all you need to know these things:

1. A docker **image** is like a blueprint of an application, and it's defined in **Dockerfiles**.
2. A docker **container** is a runnable instance of the application, built from the instructions in the blueprint.
3. You can configure your application in a `docker-compose.yml` file.

### Our Dockerfile
The first one may sound complicated, as the contents of our Dockerfile show, this can only take one line of code (from us). That's possible because Dockerfiles can build an image based on another image, and the developers of many open source projects (e.g. Ubuntu, PostgreSQL/PostGIS, MySQL, Go, nginx, etc) public official images on [Docker Hub](https://registry.hub.docker.com/search?q=). Our Dockerfile pulls the [postgis/postgis image](https://registry.hub.docker.com/r/postgis/postgis) (translated, it pulls the `postgis` base image from the `postgis` organization), and then adds nothing else. So in one line, we indicate that we want the `postgis/postgis` image that has the tag "15-3.3" (which they've intuitively used to indicate the image provides a PostGIS database that has version 15-3.3).

In [2]:
#| code-fold: true
#| code-summary: The full contents of our Dockerfile

!cat db_context/Dockerfile

FROM postgis/postgis:15-3.3

### Our docker-compose.yml file

And this is our `docker-compose.yml` file. The `docker-compose.yml` file defines the services, and any networks, volumes, configs, and/or secrets/environment variables your system needs to work. The file below defines one service (named `postgis`) and one volume (named `sandbox_postgis_data`).

In [3]:
#| code-fold: true
#| code-summary: The contents of our docker-compose.yml

!cat docker-compose.yml

version: '3.9'

services:
  postgis:
    image: sandbox_postgis:15.3.3
    build:
      context: ./db_context
      dockerfile: Dockerfile
    ports:
      - 54321:5432
    environment:
      POSTGRES_DB: db_name
      POSTGRES_USER: db_username
      POSTGRES_PASSWORD: db_password
    volumes:
      - sandbox_postgis_data:/var/lib/postgresql/data

volumes:
  sandbox_postgis_data:


There are 3 top level elements in this file: `version`, `services`, and `volumes`.

* `version` indicates the version of the Docker Compose specification; I don't think I've ever set it to anything other than 3.9.
* `services` defines configurations for each container your system needs.
* `volumes` defines persistant data stores that can be shared by different services.

#### Services
The `postgis` service has five elements: `image`, `build`, `ports`, `environment`, and `volumes`.

The `image` and `build` elements define the docker image to use; `image` defines both the name ("sandbox_postgis") and tag ("15.5.3") for the docker image, and `build` defines the Dockerfile to build into an image as well as the context to build into the docker image.

The `port` element defines a connection from a port on the host machine (host post 54321) to a port into the container (container port 5432, the default for PostgreSQL). We'll use that later to connect to the database.

The `volumes` element (in the `postgis` service) defines a persistant storage volume that will hold the data in our database. Without this, our database would reset every time we restart the system [^2].

And the `environment` element enables you to set environment variables in the container. Here, we pass in environment variables `POSTGRES_DB`, `POSTGRES_USER`, and `POSTGRES_PASSWORD` which are used to name the database (in this case, it's named `db_name`) and create a superuser (in this case having username `db_username` and password `db_password`) when the database is first created in a new volume.

### Building the image(s) used in our system

The first time you build the image defined by your Dockerfile, docker will read your Dockerfile(s), download all layers of the base image(s) (defined in lines starting with `FROM `), process each subsequent instruction into a layer, cache layers, and then bind the layers into an image. This is a template for creating containers.

The second time you run this command (assuming no changes have been made to the `docker-compose.yml` file, Dockerfile(s), or any other files the Dockerfile references), all layers will just be pulled from cache, producing much smaller output (like what's shown below).

In [4]:
#| code-fold: true
#| code-summary: The output produced while building our images

!docker compose build

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/4)                                                         
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B                                            0.0s
 => [postgis internal] load metadata for docker.io/postgis/postgis:15-3.3  0.1s
 => [postgis auth] postgis/postgis:pull token for registry-1.docker.io     0.0s
[+] Building 0.3s (3/4)                                                         
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B      

In [38]:
#| code-fold: true
#| code-summary: Shows that our system's one container is not running yet

!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


`docker ps` shows running containers, and the `-f name=...` option allows us to filter to running containers with a name containing the entered string. Currently, "sandbox-postgis" isn't part of the name of any running container, so let's spin one up.

In [39]:
#| code-fold: false
#| code-summary: The command to start up our system

!docker compose up -d

[+] Running 1/0
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠋ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.0s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠙ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.1s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠹ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.2s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠸ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.3s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠼ Container 015_docker_postgres_sandbox-postgis-1  Creating               0.4s 
[+] Running 1/2
 ✔ Network 015_docker_postgres_sandbox_default      Created                0.1s 
 ⠴ Container 

In [40]:
#| code-fold: true
#| code-summary: Shows that our system's one container is running now

!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE                    COMMAND                  CREATED         STATUS                  PORTS                                         NAMES
8513cc1ff07d   sandbox_postgis:15.3.3   "docker-entrypoint.s…"   2 seconds ago   Up Less than a second   0.0.0.0:54321->5432/tcp, :::54321->5432/tcp   015_docker_postgres_sandbox-postgis-1


## Sending commands to our database

Now we have our database up and running on our local system, but now we need to connect to it. There are countless ways to connect to databases, but I want to focus on the SQL, so I've written just enough to allow me to connect to our datasbase (via my `get_db_connection()` function) and execute a SQL query/statement (via my `execute_query()` function.

Note that `get_db_connection()` uses the `POSTGRES_DB`, `POSTGRES_USER`, and `POSTGRES_PASSWORD` environment variables as well as port number 54321, all of which we set in our `docker-compose.yml` file.

In [235]:
#| code-fold: false
#| code-summary: Our thin python-to-postgres connector code

from contextlib import contextmanager
from typing import Tuple, Union

import pandas as pd
import psycopg2


@contextmanager
def get_db_connection():
    conn = psycopg2.connect(
        dbname="db_name",
        user="db_username",
        password="db_password",
        host="localhost",
        port=54321
    )
    conn.autocommit = True
    try:
        yield conn
    finally:
        conn.close()

def execute_query_w_existing_conn(
    query: str, conn: psycopg2.extensions.connection
) -> Tuple[str, Union[pd.DataFrame, None]]:
    with conn.cursor() as cur:
        cur.execute(query)
        try:
            rows = cur.fetchall()
            col_names = [desc[0] for desc in cur.description]
            return (cur.statusmessage, pd.DataFrame(rows, columns=col_names))
        except psycopg2.ProgrammingError as err:
            if "no results to fetch" in str(err):
                return (cur.statusmessage, None)
            else:
                raise psycopg2.ProgrammingError(err)

def execute_query(
    query: str, conn: Optional[psycopg2.extensions.connection] = None
) -> Union[pd.DataFrame, None]:
    if conn is None:
        with get_db_connection() as new_conn:
            return execute_query_w_existing_conn(query=query, conn=new_conn)
    else:
        return execute_query_w_existing_conn(query=query, conn=conn)

def show_transaction_results(result: Tuple[str, str, Union[pd.DataFrame, None]]) -> None:
    query, status, result_df = result
    formatted_query = query.replace("\n", "\n    ")
    print(f"""    {formatted_query}""")
    print(f"{status}\n")
    if result_df is not None:
        print(f"records in result: {len(result_df)}\n")
        display(result_df.head())

def execute_transaction(query: str, print_results: bool = True) -> Tuple[str, str, Union[pd.DataFrame, None]]:
    queries = [el.strip() for el in query.split(";") if el.strip() != ""]
    with get_db_connection() as conn:
        results = []
        for q in queries:
            status_msg, result_df = execute_query(query=q, conn=conn)
            result = (q, status_msg, result_df)
            results.append(result)
            if print_results:
                show_transaction_results(result=result)
    return results

### Learning exercise

Imagine you're designing a data warehousing platform and you want to let other analysts or data scientists query data in the warehouse. Some users need to be able to update records or insert new records, but others only need to be able to view some datasets.

PostgreSQL has a concept called [Roles](https://www.postgresql.org/docs/15/user-manag.html) that enables you to define permissions for users or groups of users. If you aren't already familiar with how roles and permissions work, you don't have a skilled database administrator looking over your work, and you only have access to a database that other systems depend on, it would be extremely intimidating to figure out the correct permissions. Our sandbox takes off that weight by eliminating the cost of mistakes.

Let's create 

In [200]:
results = execute_transaction(
    query="""
        DROP SCHEMA raw_data CASCADE;
        DROP SCHEMA clean_data CASCADE;
    """,
    print_results=True,
)

    DROP SCHEMA raw_data CASCADE

DROP SCHEMA

    DROP SCHEMA clean_data CASCADE

DROP SCHEMA



In [201]:
#| code-fold: fold
#| code-summary: Setting up some test data tables in a raw_data schema

results = execute_transaction(
    query="""
        CREATE SCHEMA raw_data;
        CREATE TABLE raw_data.customers (
            customer_id int PRIMARY KEY,
            customer_name text,
            contact_name text,
            country text,
            email text
        );
        CREATE TABLE raw_data.orders (
            order_id int PRIMARY KEY,
            customer_id int,
            order_date text,
            product text,
            quantity int,
            FOREIGN KEY (customer_id) REFERENCES raw_data.customers(customer_id)
        );
        INSERT INTO raw_data.customers (customer_id, customer_name, contact_name, country, email)
        VALUES
            (1, 'Customer A', 'Contact A', 'Country A', 'contactA@email.com'),
            (2, 'Customer B', 'Contact B', 'Country B', 'contactB@email.com'),
            (3, 'Customer C', 'Contact C', 'Country C', 'contactC@email.com');
        INSERT INTO raw_data.orders (order_id, customer_id, order_date, product, quantity)
        VALUES
            (1, 1, '2023-01-01', 'Product A', 10),
            (2, 1, '2023-01-02', 'Product B', 15),
            (3, 2, '2023-02-01', 'Product A', 5),
            (4, 3, '2023-02-02', 'Product C', 20);
    """,
    print_results=True,
)

    CREATE SCHEMA raw_data

CREATE SCHEMA

    CREATE TABLE raw_data.customers (
                customer_id int PRIMARY KEY,
                customer_name text,
                contact_name text,
                country text,
                email text
            )

CREATE TABLE

    CREATE TABLE raw_data.orders (
                order_id int PRIMARY KEY,
                customer_id int,
                order_date text,
                product text,
                quantity int,
                FOREIGN KEY (customer_id) REFERENCES raw_data.customers(customer_id)
            )

CREATE TABLE

    INSERT INTO raw_data.customers (customer_id, customer_name, contact_name, country, email)
            VALUES
                (1, 'Customer A', 'Contact A', 'Country A', 'contactA@email.com'),
                (2, 'Customer B', 'Contact B', 'Country B', 'contactB@email.com'),
                (3, 'Customer C', 'Contact C', 'Country C', 'contactC@email.com')

INSERT 0 3

    INSERT INTO raw_data.o

In [202]:
#| code-fold: fold
#| code-summary: Creating a clean_data schema and cleaning raw data

results = execute_transaction(
    query="""
        CREATE SCHEMA clean_data;
        CREATE TABLE clean_data.customers (
            customer_id int PRIMARY KEY,
            customer_name text,
            contact_name text,
            country text,
            email text
        );
        CREATE TABLE clean_data.orders (
            order_id int PRIMARY KEY,
            customer_id int,
            order_date date,
            product text,
            quantity int,
            FOREIGN KEY (customer_id) REFERENCES clean_data.customers(customer_id)
        );
        INSERT INTO clean_data.customers (customer_id, customer_name, contact_name, country, email)
        SELECT
            customer_id,
            customer_name,
            contact_name,
            country,
            lower(email) AS email
        FROM raw_data.customers;
        INSERT INTO clean_data.orders (order_id, customer_id, order_date, product, quantity)
        SELECT
            order_id,
            customer_id,
            order_date::date AS order_date,
            product,
            quantity
        FROM raw_data.orders;
    """,
    print_results=True,
)

    CREATE SCHEMA clean_data

CREATE SCHEMA

    CREATE TABLE clean_data.customers (
                customer_id int PRIMARY KEY,
                customer_name text,
                contact_name text,
                country text,
                email text
            )

CREATE TABLE

    CREATE TABLE clean_data.orders (
                order_id int PRIMARY KEY,
                customer_id int,
                order_date date,
                product text,
                quantity int,
                FOREIGN KEY (customer_id) REFERENCES clean_data.customers(customer_id)
            )

CREATE TABLE

    INSERT INTO clean_data.customers (customer_id, customer_name, contact_name, country, email)
            SELECT
                customer_id,
                customer_name,
                contact_name,
                country,
                lower(email) AS email
            FROM raw_data.customers

INSERT 0 3

    INSERT INTO clean_data.orders (order_id, customer_id, order_date, produ

In [205]:
#| code-fold: show
#| code-summary: Checking that my role can see a table

results = execute_transaction(
    query="""
        SELECT current_user;

        SELECT * FROM clean_data.customers;
    """,
    print_results=True,
)

    SELECT current_user

SELECT 1

records in result: 1



,current_user
0,db_username


    SELECT * FROM clean_data.customers

SELECT 3

records in result: 3



,customer_id,customer_name,contact_name,country,email
0,1,Customer A,Contact A,Country A,contacta@email.com
1,2,Customer B,Contact B,Country B,contactb@email.com
2,3,Customer C,Contact C,Country C,contactc@email.com


In [238]:
#| code-fold: show
#| code-summary: Creating a new role and seeing what it can see before granting it permissions

results = execute_transaction(
    query="""
        DROP ROLE data_analyst;

            """,
    print_results=True,
)

    DROP ROLE data_analyst
DROP ROLE



In [239]:
#| code-fold: show
#| code-summary: Creating a new role and seeing what it can see before granting it permissions

results = execute_transaction(
    query="""
        CREATE ROLE data_analyst;

        SET ROLE data_analyst;

        SELECT current_user;

        SELECT * FROM clean_data.customers;
    """,
    print_results=True,
)

    CREATE ROLE data_analyst
CREATE ROLE

    SET ROLE data_analyst
SET

    SELECT current_user
SELECT 1

records in result: 1



,current_user
0,data_analyst


InsufficientPrivilege: permission denied for schema clean_data
LINE 1: SELECT * FROM clean_data.customers
                      ^


Good. We created that role, switched into that role, confirmed we were in the `data_analyst` role, and then were stopped from accessing a table we didn't have permission to view.

The error message also helps us see what we don't have permission to access: the `clean_data` schema.

A few privileges have to be granted before the `data_analyst` role can view that table.

In [244]:
#| code-fold: show
#| code-summary: Creating a new role and seeing what it can see before granting it permissions

try:
    results = execute_transaction(
        query="""
            GRANT USAGE ON SCHEMA clean_data TO data_analyst;
            SET ROLE data_analyst;
            SELECT current_user;
            SELECT * FROM clean_data.customers;
        """,
        print_results=True,
    )
except psycopg2.errors.InsufficientPrivilege as err:
    print(f"InsufficientPrivilege Error: {err}")

    GRANT USAGE ON SCHEMA clean_data TO data_analyst
GRANT

    SET ROLE data_analyst
SET

    SELECT current_user
SELECT 1

records in result: 1



,current_user
0,data_analyst


InsufficientPrivilege Error: permission denied for table customers



In [245]:
#| code-fold: show
#| code-summary: Altering the default permissions for any user granted role data_analyst in the future

try:
    results = execute_transaction(
        query="""
            ALTER DEFAULT PRIVILEGES IN SCHEMA clean_data
                GRANT SELECT ON TABLES TO data_analyst;

            SET ROLE data_analyst;
            SELECT current_user;
            SELECT * FROM clean_data.customers;
        """,
        print_results=True,
    )
except psycopg2.errors.InsufficientPrivilege as err:
    print(f"InsufficientPrivilege Error: {err}")

    ALTER DEFAULT PRIVILEGES IN SCHEMA clean_data
                    GRANT SELECT ON TABLES TO data_analyst
ALTER DEFAULT PRIVILEGES

    SET ROLE data_analyst
SET

    SELECT current_user
SELECT 1

records in result: 1



,current_user
0,data_analyst


InsufficientPrivilege Error: permission denied for table customers



In [246]:
#| code-fold: show
#| code-summary: Granting the data_analyst role privileges to SELECT from tables in the clean_data schema

try:
    results = execute_transaction(
        query="""
            GRANT SELECT ON ALL TABLES IN SCHEMA clean_data TO data_analyst;

            SET ROLE data_analyst;
            SELECT current_user;
            SELECT * FROM clean_data.customers;
        """,
        print_results=True,
    )
except psycopg2.errors.InsufficientPrivilege as err:
    print(f"InsufficientPrivilege Error: {err}")

    GRANT SELECT ON ALL TABLES IN SCHEMA clean_data TO data_analyst
GRANT

    SET ROLE data_analyst
SET

    SELECT current_user
SELECT 1

records in result: 1



,current_user
0,data_analyst


    SELECT * FROM clean_data.customers
SELECT 3

records in result: 3



,customer_id,customer_name,contact_name,country,email
0,1,Customer A,Contact A,Country A,contacta@email.com
1,2,Customer B,Contact B,Country B,contactb@email.com
2,3,Customer C,Contact C,Country C,contactc@email.com


It took all three steps to grant the `data_analyst` role sufficient privileges to view the table.

TODO:
* Create a user role,
* Show that the user can't view the table,
* Grant the user that data analyst role,
* Show it gained the privileges,
* Revoke privs,
* Show revocation

In [184]:
#| code-fold: show
#| code-summary: Checking 

msg, pg_roles_df = execute_query(
    query="""
        SELECT rolname, rolsuper, rolcreaterole, rolcreatedb
        FROM pg_roles
        WHERE rolname = 'data_analyst';
    """
)
pg_roles_df

,rolname,rolsuper,rolcreaterole,rolcreatedb
0,pg_database_owner,False,False,False
1,pg_read_all_data,False,False,False
2,pg_write_all_data,False,False,False
3,pg_monitor,False,False,False
4,pg_read_all_settings,False,False,False
5,pg_read_all_stats,False,False,False
6,pg_stat_scan_tables,False,False,False
7,pg_read_server_files,False,False,False
8,pg_write_server_files,False,False,False
9,pg_execute_server_program,False,False,False


In [185]:
execute_query(query="""CREATE ROLE data_analyst;""")

('CREATE ROLE', None)

In [186]:
msg, pg_roles_df = execute_query(
    query="""
        SELECT rolname, rolsuper, rolcreaterole, rolcreatedb
        FROM pg_roles;
    """
)
pg_roles_df

,rolname,rolsuper,rolcreaterole,rolcreatedb
0,pg_database_owner,False,False,False
1,pg_read_all_data,False,False,False
2,pg_write_all_data,False,False,False
3,pg_monitor,False,False,False
4,pg_read_all_settings,False,False,False
5,pg_read_all_stats,False,False,False
6,pg_stat_scan_tables,False,False,False
7,pg_read_server_files,False,False,False
8,pg_write_server_files,False,False,False
9,pg_execute_server_program,False,False,False


In [103]:
execute_query(query="""SET ROLE data_analyst;""")

('SET', None)

In [166]:
result = execute_query(query="""RESET ROLE;""")
result

('RESET', None)

In [173]:
_ = execute_transaction(query="""SELECT current_user;""")

Query: 
    SELECT current_user
Status msg: SELECT 1

records in result: 1



,current_user
0,db_username


In [175]:
_ = execute_transaction(query="""DROP ROLE data_analyst;""")

Query: 
    DROP ROLE data_analyst
Status msg: DROP ROLE



In [164]:
execute_query(query="""SELECT current_user;""")

('SELECT 1',
   current_user
 0  db_username)

In [85]:
schemas = execute_query(query="""
    SELECT nspname AS schema_name
    FROM pg_namespace;
""")
schemas

,schema_name
0,pg_toast
1,pg_catalog
2,public
3,information_schema
4,topology
5,tiger
6,tiger_data


In [ ]:
pg_roles_df = execute_query(
    query="""
        SELECT rolname, rolsuper, rolcreaterole, rolcreatedb
        FROM pg_roles;
    """
)
pg_roles_df

,rolname,rolsuper,rolcreaterole,rolcreatedb
0,pg_database_owner,False,False,False
1,pg_read_all_data,False,False,False
2,pg_write_all_data,False,False,False
3,pg_monitor,False,False,False
4,pg_read_all_settings,False,False,False
5,pg_read_all_stats,False,False,False
6,pg_stat_scan_tables,False,False,False
7,pg_read_server_files,False,False,False
8,pg_write_server_files,False,False,False
9,pg_execute_server_program,False,False,False


In [ ]:
pg_databases_df = execute_query(
    query="""
        SELECT *
        FROM pg_database;
    """
)
pg_databases_df

,oid,datname,datdba,encoding,datlocprovider,datistemplate,datallowconn,datconnlimit,datfrozenxid,datminmxid,dattablespace,datcollate,datctype,daticulocale,datcollversion,datacl
0,5,postgres,10,6,c,False,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None
1,16384,db_name,10,6,c,False,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None
2,1,template1,10,6,c,True,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,"{=c/db_username,db_username=CTc/db_username}"
3,4,template0,10,6,c,True,False,-1,717,1,1663,en_US.utf8,en_US.utf8,None,None,"{=c/db_username,db_username=CTc/db_username}"
4,16385,template_postgis,10,6,c,True,True,-1,717,1,1663,en_US.utf8,en_US.utf8,None,2.31,None


In [ ]:
pg_namespaces_df = execute_query(
    query="""
        SELECT *
        FROM pg_namespace;
    """
)
pg_namespaces_df

,oid,nspname,nspowner,nspacl
0,99,pg_toast,10,None
1,11,pg_catalog,10,"{db_username=UC/db_username,=U/db_username}"
2,2200,public,6171,"{pg_database_owner=UC/pg_database_owner,=U/pg_..."
3,13186,information_schema,10,"{db_username=UC/db_username,=U/db_username}"
4,19052,topology,10,"{db_username=UC/db_username,=U/db_username}"
5,19226,tiger,10,None
6,19482,tiger_data,10,None


In [ ]:
pg_data_types_df = execute_query(
    query="""
        SELECT
            typname, typlen, typispreferred, typcategory, typtype,
            typinput, typoutput, typreceive, typsend
        FROM pg_type;
    """
)
pg_data_types_df

,typname,typlen,typispreferred,typcategory,typtype,typinput,typoutput,typreceive,typsend
0,bool,1,True,B,b,boolin,boolout,boolrecv,boolsend
1,bytea,-1,False,U,b,byteain,byteaout,bytearecv,byteasend
2,char,1,False,Z,b,charin,charout,charrecv,charsend
3,name,64,False,S,b,namein,nameout,namerecv,namesend
4,int8,8,False,N,b,int8in,int8out,int8recv,int8send
...,...,...,...,...,...,...,...,...,...
714,_pagc_gaz,-1,False,A,b,array_in,array_out,array_recv,array_send
715,pagc_lex,-1,False,C,c,record_in,record_out,record_recv,record_send
716,_pagc_lex,-1,False,A,b,array_in,array_out,array_recv,array_send
717,pagc_rules,-1,False,C,c,record_in,record_out,record_recv,record_send


In [ ]:
pg_data_types_df = execute_query(
    query="""
        SELECT *
        FROM pg_type
        WHERE typcategory = 'D';
    """
)
pg_data_types_df

,oid,typname,typnamespace,typowner,typlen,typbyval,typtype,typcategory,typispreferred,typisdefined,typdelim,typrelid,typsubscript,typelem,typarray,typinput,typoutput,typreceive,typsend,typmodin,typmodout,typanalyze,typalign,typstorage,typnotnull,typbasetype,typtypmod,typndims,typcollation,typdefaultbin,typdefault,typacl
0,1082,date,11,10,4,True,b,D,False,True,",",0,-,0,1182,date_in,date_out,date_recv,date_send,-,-,-,i,p,False,0,-1,0,0,None,None,None
1,1083,time,11,10,8,True,b,D,False,True,",",0,-,0,1183,time_in,time_out,time_recv,time_send,timetypmodin,timetypmodout,-,d,p,False,0,-1,0,0,None,None,None
2,1114,timestamp,11,10,8,True,b,D,False,True,",",0,-,0,1115,timestamp_in,timestamp_out,timestamp_recv,timestamp_send,timestamptypmodin,timestamptypmodout,-,d,p,False,0,-1,0,0,None,None,None
3,1184,timestamptz,11,10,8,True,b,D,True,True,",",0,-,0,1185,timestamptz_in,timestamptz_out,timestamptz_recv,timestamptz_send,timestamptztypmodin,timestamptztypmodout,-,d,p,False,0,-1,0,0,None,None,None
4,1266,timetz,11,10,12,False,b,D,False,True,",",0,-,0,1270,timetz_in,timetz_out,timetz_recv,timetz_send,timetztypmodin,timetztypmodout,-,d,p,False,0,-1,0,0,None,None,None
5,13211,time_stamp,13186,10,8,True,d,D,False,True,",",0,-,0,13210,domain_in,timestamptz_out,domain_recv,timestamptz_send,-,-,-,d,p,False,1184,2,0,0,{SQLVALUEFUNCTION :op 4 :type 1184 :typmod 2 :...,CURRENT_TIMESTAMP(2),None


In [ ]:
pg_data_types_df.loc[pg_data_types_df["typname"].str.contains("geometry")]

,typname,typlen,typispreferred,typcategory,typtype,typinput,typoutput,typreceive,typsend
613,geometry,-1,False,U,b,geometry_in,geometry_out,geometry_recv,geometry_send
614,_geometry,-1,False,A,b,array_in,array_out,array_recv,array_send
623,geometry_dump,-1,False,C,c,record_in,record_out,record_recv,record_send
624,_geometry_dump,-1,False,A,b,array_in,array_out,array_recv,array_send
633,geometry_columns,-1,False,C,c,record_in,record_out,record_recv,record_send
634,_geometry_columns,-1,False,A,b,array_in,array_out,array_recv,array_send
639,topogeometry,-1,False,C,c,record_in,record_out,record_recv,record_send
640,_topogeometry,-1,False,A,b,array_in,array_out,array_recv,array_send


In [ ]:
pg_data_types_df["typcategory"].value_counts(dropna=False)

typcategory
A    348
C    253
U     27
P     25
N     20
R     12
Z      7
S      7
G      7
D      6
I      2
V      2
B      1
X      1
T      1
Name: count, dtype: int64

* A: Array types
* B: Boolean types
* C: Composite types
* D: Date/time types
* E: Enum types
* G: Geometric types
* I: Network address types
* N: Numeric types
* P: Pseudo-types
* R: Range types
* S: String types
* T: Text search types
* U: UUID types
* V: Bit-string types
* X: Unknown type

In [46]:
pg_data_types_df.loc[pg_data_types_df["typcategory"] == "U"]

,typname,typlen,typispreferred,typcategory,typtype,typinput,typoutput,typreceive,typsend
1,bytea,-1,False,U,b,byteain,byteaout,bytearecv,byteasend
11,tid,6,False,U,b,tidin,tidout,tidrecv,tidsend
12,xid,4,False,U,b,xidin,xidout,xidrecv,xidsend
13,cid,4,False,U,b,cidin,cidout,cidrecv,cidsend
19,json,-1,False,U,b,json_in,json_out,json_recv,json_send
20,xml,-1,False,U,b,xml_in,xml_out,xml_recv,xml_send
26,xid8,8,False,U,b,xid8in,xid8out,xid8recv,xid8send
38,macaddr,6,False,U,b,macaddr_in,macaddr_out,macaddr_recv,macaddr_send
41,macaddr8,8,False,U,b,macaddr8_in,macaddr8_out,macaddr8_recv,macaddr8_send
42,aclitem,12,False,U,b,aclitemin,aclitemout,-,-


In [46]:
pg_data_types_df.loc[pg_data_types_df["typcategory"] == "U"]

,typname,typlen,typispreferred,typcategory,typtype,typinput,typoutput,typreceive,typsend
1,bytea,-1,False,U,b,byteain,byteaout,bytearecv,byteasend
11,tid,6,False,U,b,tidin,tidout,tidrecv,tidsend
12,xid,4,False,U,b,xidin,xidout,xidrecv,xidsend
13,cid,4,False,U,b,cidin,cidout,cidrecv,cidsend
19,json,-1,False,U,b,json_in,json_out,json_recv,json_send
20,xml,-1,False,U,b,xml_in,xml_out,xml_recv,xml_send
26,xid8,8,False,U,b,xid8in,xid8out,xid8recv,xid8send
38,macaddr,6,False,U,b,macaddr_in,macaddr_out,macaddr_recv,macaddr_send
41,macaddr8,8,False,U,b,macaddr8_in,macaddr8_out,macaddr8_recv,macaddr8_send
42,aclitem,12,False,U,b,aclitemin,aclitemout,-,-


[^1] Before you can use docker and compose, you have to install [docker](https://docs.docker.com/engine/install/) and [compose](https://docs.docker.com/compose/install/). I installed the docker engine and compose, but it looks like the developers at Docker Inc. guide people towards installing their Docker Desktop client. In either setup, docker should still become available to you via the command line, so instructions in this post should work.

[^2] Docker containers are designed to be a replicable instance of an image, so when you shut down your application, your containers are removed and new ones are created next time you start it up. This is great for reproducibility (you always get a new, clean instance based on your image), but you don't want the data you ingest into your database to get wiped every time you shut down your system, so you can define a persistent **volume** that will live on in the host system.